In [ ]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay
!pip install piglet

ACTION  
*   0 = south -> -1
*   1 = north -> -1
*   2 = east -> -1
*   3 = west -> -1
*   4 = pickup -> -10, incorrect location
*   5 = dropoff  -> -10, incorrect location, +20, correct location




In [ ]:
import gym

env = gym.make("Taxi-v3").env # this last .env avoids training stopping at 200 iterations, which is the default for the new version of Gym

state = env.encode(4, 3, 1, 0) #taxi row, taxi column, passeger's location, passenger's destination
print(f'State {state}')
env.s = state
env.render()

#env.reset(), reset the environment and return to random state
#env.step(action), take an action or step the environment by one timestep -> return: observation, return, done, info

print(f'Action space is {env.action_space}')
print(f'Observation space is {env.observation_space}')



State 464
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space is Discrete(6)
Observation space is Discrete(500)


In [ ]:
import gym

env = gym.make("Taxi-v3").env

env.s = 105
env.render()

print(f'Action space is {env.action_space}')
print(f'Observation space is {env.observation_space}')



+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space is Discrete(6)
Observation space is Discrete(500)


In [ ]:
env.P[328] #the object P is a matrix of state / action and saves the reward values assigned for this specific pair.

gym.envs.toy_text.taxi.TaxiEnv

In [ ]:
env.P[328][0] #{action: [(probability, nextstate, reward, done)]}

[(1.0, 428, -1, False)]

## *Simulate with brute force approach*

In [ ]:
env.s = 328

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 458
Penalties incurred: 135


In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, step_time = .1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(step_time)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 63
State: 388
Action: 2
Reward: -1


KeyboardInterrupt: ignored

##  *Training Q-Learning*

In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [ ]:
import random
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.39
Average penalties per episode: 0.0


## *Visualize Q-Learning*



In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

frames_q = [] # for animation

state = env.reset()
epochs, penalties, reward = 0, 0, 0
    
done = False
    
while not done:

  action = np.argmax(q_table[state])
  state, reward, done, info = env.step(action)
  frames_q.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
  )
  if reward == -10:
    penalties -= 10

  epochs += 1

  total_penalties += penalties
  total_epochs += epochs

print(f"Penalties: {penalties}")

Penalties: 0


In [ ]:
from IPython.display import clear_output
from time import sleep
print_frames(frames_q, 0.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 18
State: 410
Action: 5
Reward: 20
